In [1]:
import os
import subprocess
import sys
import numpy as np
import itertools
import json
from tqdm import tqdm
from IPython.display import display, HTML
import time
import copy
from collections import namedtuple
from functools import partial
top_level_dir = '/'.join( os.getcwd().split( '/' )[ :-2 ] )
if top_level_dir not in sys.path:
    sys.path.append( top_level_dir )
%load_ext autoreload
%autoreload 2
display(HTML("<style>.container { width:100% !important; }</style>"))

### This notebook will look at the convergence rate for different kinds of graphs

In [30]:
from GenModels.GM.Distributions import Categorical, Dirichlet, TensorTransition, TensorTransitionDirichletPrior
from GenModels.GM.Models.DiscreteGraphModels import *
from GenModels.GM.States.GraphicalMessagePassing import DataGraph

In [31]:
graphLine = DataGraph()
for i in range( 1, 100 ):
    graphLine.addEdge( parents=[ i-1 ], children=[ i ] )
graphs = [ ( graphLine, np.array( [] ) ) ]

# Test 1 - Deep graphs without cycles

In [32]:
initial_shape, transition_shapes, emission_shape = GHMM.parameterShapes( graphs, d_latent=3, d_obs=4 )
initial_priors = np.ones( initial_shape )
transition_priors = [ np.ones( shape ) for shape in transition_shapes ]
emission_prior = np.ones( emission_shape )

In [33]:
trueModel = GHMM( graphs, priors=( initial_priors, transition_priors, emission_prior ), method='EM' )

In [34]:
trueModel.fitStep()

AssertionError: If there is only 1 measurement, add an extra dim!

# Test 2 - Wide graphs without cycles

# Test 3 - Deep graphs with cycles

# Test 4 - Wide graphs with cycles